# Project Milestone 1

## 1. Loading of the data and storing of vaccination dataframe

In [1]:
import pandas as pd

PATH = 'Data/'

In [3]:

def process_chunk(chunk):
    vaccination_quotes = chunk[chunk['quotation'].str.contains("vaccin")]
    return vaccination_quotes

vaccination_df = pd.DataFrame()

for i in range(20,21): #(15,21)
    DATA_FILE = 'quotes-20{}.json.bz2'.format(i)
    with pd.read_json(PATH + DATA_FILE, lines=True, compression='bz2', chunksize=100000) as df_reader:
        for chunk in df_reader:
            vaccination_df = pd.concat([vaccination_df, process_chunk(chunk)])

vaccination_df.shape
vaccination_df.to_pickle("Data/2020_vaccin.pck")         
chunk.to_pickle("Data/small_df.pck")

In [4]:
VACCINATION_DATASET = PATH+"2020_vaccin.pck"
vaccination = pd.read_pickle(VACCINATION_DATASET)
vaccination

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
317,2020-04-14-009511,comprehensive AIDS vaccine research initiative,President Clinton,[Q1124],2020-04-14 06:54:55,1,"[[President Clinton, 0.9128], [None, 0.0783], ...",[https://newswithviews.com/dr-fauci-and-dr-bir...,E
1550,2020-04-11-023226,"Some react fine to vaccines, others are paraly...",Mary Holland,"[Q28957276, Q6779784]",2020-04-11 08:25:08,3,"[[Mary Holland, 0.889], [None, 0.111]]",[https://www.freemalaysiatoday.com/category/wo...,E
2312,2020-03-02-074888,"Well, we've asked them to accelerate whatever ...",President Donald Trump,[Q22686],2020-03-02 17:19:01,1,"[[President Donald Trump, 0.6776], [None, 0.32...",[https://www.zawya.com/mena/en/multimedia/vide...,E
3596,2020-02-27-039401,"It is not a race between vaccine producers, it...",Paul Young,"[Q22101574, Q360501, Q3898174, Q46110256, Q518...",2020-02-27 21:53:39,1,"[[Paul Young, 0.9398], [None, 0.0602]]",[https://www.newshub.co.nz/home/world/2020/02/...,E
5597,2020-01-16-027502,I am only hopeful that when we start studying ...,None,[],2020-01-16 16:50:13,1,"[[None, 0.8717], [Melinda Gates, 0.1283]]",[http://www.medscape.com/viewarticle/923795?sr...,E
...,...,...,...,...,...,...,...,...,...
5242097,2020-03-11-009813,called for more public education about the vir...,Hillary Clinton,[Q6294],2020-03-11 15:28:51,2,"[[Hillary Clinton, 0.8098], [None, 0.1018], [D...",[http://www.politifact.com/factchecks/2020/mar...,E
5242130,2020-03-11-011441,"(COVID-19 is) different because, by definition...",Steve Woodard,[Q7614354],2020-03-11 19:53:44,1,"[[Steve Woodard, 0.7567], [None, 0.2433]]",[http://news4jax.com/news/local/2020/03/11/cit...,E
5242698,2020-04-12-012059,If we're going to have economic distress until...,Neel Kashkari,[Q1287635],2020-04-12 18:00:13,8,"[[Neel Kashkari, 0.8851], [None, 0.1149]]",[http://feeds.cbsnews.com/~r/CBSNewsMain/~3/hw...,E
5243373,2020-02-03-062993,Research continues on other approaches to a sa...,Anthony Fauci,[Q573246],2020-02-03 00:00:00,40,"[[Anthony Fauci, 0.8352], [None, 0.1195], [Lin...",[http://webwire.com/ViewPressRel.asp?aId=25435...,E
